In [ ]:
!apt-get install swig -y  # c or c++ -> python, ruby , etc ...
!pip install Cython numpy # 파이썬 코드를 C언어로 컴파일함
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip3 install
!sudo apt-get install build-essential swig

!pip install auto-sklearn==0.10.0

In [ ]:
!pip install autokeras

In [ ]:
# ignore some annoying warnings for demonstrating auto-sklearn 
import numpy as np
np.warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd MyDrive/Colab Notebooks/AI교육/3주차/미니프로젝트

In [ ]:
import os
import time
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
import logging
from warnings import simplefilter
import autosklearn.regression

##데이터 불러오기

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI교육/3주차/미니프로젝트/insurance.csv')
train_data.head()

In [ ]:
print(train_data.dtypes)

##전처리

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

#문자 데이터 숫자로 변환
train_data["smoker"] = label_encoder.fit_transform(train_data["smoker"])
train_data["sex"] = label_encoder.fit_transform(train_data["sex"])
train_data["region"] = label_encoder.fit_transform(train_data["region"])


In [ ]:
print(train_data)

In [ ]:
target = train_data['charges']
train_data.drop(['charges'], inplace=True, axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data, target, test_size = 0.2, random_state = 42, shuffle=False)

In [ ]:
print(f'Sizes: X_train={X_train.shape}, y_train={y_train.shape}, X_test={X_test.shape}, y_test={y_test.shape}')

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
import numpy as np
import tensorflow as tf

import autokeras as ak

In [ ]:
Data_x = X_train.to_numpy()

In [ ]:
Data_x

In [ ]:
Data_y = y_train.to_numpy()
Data_y

In [ ]:
import autosklearn.regression

In [ ]:
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=120,
    per_run_time_limit=30,
)

automl.fit(X_train, y_train)

In [ ]:
print(automl.show_models())

In [ ]:
import sklearn

predictions = automl.predict(X_test)
print("R2 score:", sklearn.metrics.r2_score(y_test, predictions))


In [ ]:

print(predictions)


In [ ]:
print(y_test)

In [ ]:
!pip install --upgrade pip --user
!pip install "mxnet_cu101<2.0.0, >=1.7.0"
!pip install autogluon

#AutoGluon

In [ ]:
from autogluon.tabular import TabularPredictor

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI교육/3주차/미니프로젝트/insurance.csv')
train_data.shape

In [ ]:
train_data=train_data.sample(frac=1,random_state=100).reset_index(drop=True)
test_data = train_data[1100:]
train_data = train_data[:1100]
train_data

In [ ]:
label = 'charges'
print("Summary of class variable: \n", train_data[label].describe())

In [ ]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path,problem_type='regression').fit(train_data)

In [ ]:
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

In [ ]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

In [ ]:
y_test

In [ ]:
predictor.leaderboard(test_data, silent=True)

In [ ]:
pred_probs = predictor.predict_proba(test_data_nolab)
pred_probs.head(5)

In [ ]:
predictor_age = TabularPredictor(label=label).fit(train_data, time_limit=60,)
performance = predictor_age.evaluate(test_data)